# Load Cora Dataset 

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = torch.device('cpu')
import numpy as np
import random

data = torch.load('data.pth')
g = data['g'].to(device)
feat = data['feat'].to(device)
label = data['label'].to(device)
train_nodes = data['train_nodes']
val_nodes = data['val_nodes']
test_nodes = data['test_nodes']

#保证固定性
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
setup_seed(20)

Using backend: pytorch


# Load victim GCN model

In [2]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    """Graph Convolution Network (GCN)
    加载模型:图卷积神经网络

    Example
    -------
    # GCN with one hidden layer
    >>> model = GCN(100, 10, hid=32)
    """
    def __init__(self,
                 in_feats: int,
                 out_feats: int,
                 hid: list = 16,
                 dropout: float = 0.5):
        super().__init__()
        #两层，分别用于接收输入和进行输出
        self.conv1 = GraphConv(in_feats, hid)
        self.conv2 = GraphConv(hid, out_feats)
        self.dropout = nn.Dropout(dropout)

    def forward(self, g, feat):

        if torch.is_tensor(g):
            feat = self.dropout(feat)
            feat = g @ (feat @ self.conv1.weight) + self.conv1.bias
            feat = F.relu(feat)
            feat = self.dropout(feat)
            feat = g @ (feat @ self.conv2.weight) + self.conv2.bias
            return feat
        
        #添加自环
        g = g.add_self_loop()
        feat = self.dropout(feat)
        feat = self.conv1(g, feat)
        feat = F.relu(feat)
        feat = self.dropout(feat)
        feat = self.conv2(g, feat)
        return feat

device = torch.device('cpu')

num_feats = feat.size(1)
num_classes = int(label.max() + 1)
model = GCN(num_feats, num_classes).to(device)

model.load_state_dict(torch.load('model.pth', map_location=device))
model.eval()


GCN(
  (conv1): GraphConv(in=1433, out=16, normalization=both, activation=None)
  (conv2): GraphConv(in=16, out=7, normalization=both, activation=None)
  (dropout): Dropout(p=0.5, inplace=False)
)

# Attack

*Goodfellow et al.* [📝Explaining and Harnessing Adversarial Examples](https://arxiv.org/abs/1412.6572), *ICLR'15*

*Chen et al.* [📝Fast Gradient Attack on Network Embedding](https://arxiv.org/abs/1809.02797), *arXiv'18*

*Chen et al.* [📝Link Prediction Adversarial Attack Via Iterative Gradient Attack](https://ieeexplore.ieee.org/abstract/document/9141291), *IEEE Trans'20* 

*Dai et al.* [📝Adversarial Attack on Graph Structured Data](https://arxiv.org/abs/1806.02371), *ICML'18*

In [3]:
from typing import Optional

import dgl
from torch.autograd import grad
from tqdm import tqdm

#计算邻接矩阵的归一化
def normalize(adj_matrix: torch.Tensor, norm: str = 'both'):

    if norm == 'none':
        return adj_matrix

    src_degrees = adj_matrix.sum(dim=0).clamp(min=1)
    dst_degrees = adj_matrix.sum(dim=1).clamp(min=1)

    if norm == 'left':
        # A * D^-1
        norm_src = (1.0 / src_degrees).view(1, -1)
        adj_matrix = adj_matrix * norm_src
    elif norm == 'right':
        # D^-1 * A
        norm_dst = (1.0 / dst_degrees).view(-1, 1)
        adj_matrix = adj_matrix * norm_dst
    else:  # both or square
        if norm == 'both':
            # D^-0.5 * A * D^-0.5
            pow = -0.5
        else:
            # D^-1 * A * D^-1
            pow = -1
        norm_src = torch.pow(src_degrees, pow).view(1, -1)
        norm_dst = torch.pow(dst_degrees, pow).view(-1, 1)
        adj_matrix = norm_src * adj_matrix * norm_dst
    return adj_matrix

class FGAttack(nn.Module):
    """FGSM"""
    def __init__(self, graph: dgl.DGLGraph, feat: torch.Tensor, 
                 surrogate: nn.Module, device: str = "cpu"):
        super().__init__()
        self.graph = graph
        self.feat = feat
        self.device = torch.device(device)
        self.surrogate = surrogate.to(self.device)
        self.loss_fn = nn.CrossEntropyLoss()

    #调用reset方法来回归攻击前的状态
    def reset(self):
        self.modified_adj = self.graph.add_self_loop().adjacency_matrix().to_dense().to(self.device)
        self.modified_feat = self.feat.clone()
        return self

    def attack(self,
               target,
               target_label,
               num_budgets,
               direct_attack=True,
               structure_attack=True,
               feature_attack=False,
               disable=False):

        target_label = torch.as_tensor(target_label, device=self.device, dtype=torch.long).view(-1)

        self.structure_attack = structure_attack
        self.feature_attack = feature_attack
        self.direct_attack = direct_attack
        
        modified_adj = self.modified_adj
        modified_feat = self.modified_feat
        modified_adj.requires_grad_(bool(structure_attack))
        modified_feat.requires_grad_(bool(feature_attack))

        target = torch.as_tensor(target, device=self.device, dtype=torch.long)
        target_label = torch.as_tensor(target_label, device=self.device, dtype=torch.long).view(-1)
        num_nodes, num_feats = modified_feat.size()

        for it in tqdm(range(num_budgets),
                       desc='Peturbing Graph',
                       disable=disable):

            adj_grad, feat_grad = self._compute_gradients(modified_adj,
                                                          modified_feat,
                                                          target, target_label)

            adj_grad_score = modified_adj.new_zeros(1)
            print(adj_grad_score.shape)
            exit()
            feat_grad_score = modified_feat.new_zeros(1)

            with torch.no_grad():
                if structure_attack:
                    adj_grad_score = self.structure_score(modified_adj,
                                                          adj_grad,
                                                          target)

                if feature_attack:
                    feat_grad_score = self.feature_score(modified_feat,
                                                         feat_grad,
                                                         target)

                adj_max, adj_argmax = torch.max(adj_grad_score, dim=0)
                feat_max, feat_argmax = torch.max(feat_grad_score, dim=0)
                if adj_max >= feat_max:
                    u, v = divmod(adj_argmax.item(), num_nodes)
                    if direct_attack:
                        u = target.item()
                    edge_weight = modified_adj[u, v].data.item()
                    modified_adj[u, v].data.fill_(1 - edge_weight)
                    modified_adj[v, u].data.fill_(1 - edge_weight)

                else:
                    u, v = divmod(feat_argmax.item(), num_feats)
                    feat_weight = modified_feat[u, v].data
                    modified_feat[u, v].data.fill_(1 - feat_weight)

        return modified_adj, modified_feat

    def structure_score(self, modified_adj, adj_grad, target):
        if self.direct_attack:
            score = adj_grad[target] * (1 - 2 * modified_adj[target])
            score -= score.min()
            # make sure the targeted node would not be selected
            score[target] = -1
        else:
            score = adj_grad * (1 - 2 * modified_adj)
            score -= score.min()
            score = torch.triu(score, diagonal=1)
            # make sure the targeted node and its neighbors would not be selected
            score[target] = -1
            score[:, target] = -1
        return score.view(-1)

    def feature_score(self, modified_feat, feat_grad, target):
        if self.direct_attack:
            score = feat_grad[target] * (1 - 2 * modified_feat[target])
        else:
            score = feat_grad * (1 - 2 * modified_feat)

        score -= score.min()
        # make sure the targeted node would not be selected
        score[target] = -1
        return score.view(-1)

    def _compute_gradients(self, modified_adj, modified_feat, target, target_label):

        adj_norm = normalize(modified_adj)
        logit = self.surrogate(adj_norm, modified_feat)[target].view(1, -1)
        loss = self.loss_fn(logit, target_label)

        if self.structure_attack and self.feature_attack:
            return grad(loss, [modified_adj, modified_feat], create_graph=False)

        if self.structure_attack:
            return grad(loss, modified_adj, create_graph=False)[0], None

        if self.feature_attack:
            return None, grad(loss, modified_feat, create_graph=False)[0]


In [4]:
target = 1
target_label = label[target]
print("target label: ", target_label)
budget = g.in_degrees(target) # set attack budget as node degree

attacker = FGAttack(g, feat, surrogate=model, device=device)
attacker.reset()
modified_adj, modified_feat = attacker.attack(target, target_label, budget)
attack_g = dgl.graph(modified_adj.nonzero(as_tuple=True))
attack_feat = modified_feat

target label:  tensor(2)


Peturbing Graph:  50%|█████     | 2/4 [00:00<00:00, 14.81it/s]

torch.Size([1])
torch.Size([1])


Peturbing Graph: 100%|██████████| 4/4 [00:00<00:00, 15.56it/s]

torch.Size([1])
torch.Size([1])


Peturbing Graph: 100%|██████████| 4/4 [00:00<00:00, 15.44it/s]


# Evaluate 

In [ ]:
# predict with raw graph
model(g, feat)[target].argmax()

: 

: 

In [ ]:
# predict with attacked graph: target gets misclassified
model(attack_g, attack_feat)[target].argmax()

# Save Attacked Graph for Defense

In [ ]:
torch.save(dict(attack_g=attack_g, attack_feat=attack_feat), 'attack_graph.pth')

In [ ]:
torch.save(model.state_dict(), 'model.pth')